In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams["figure.dpi"] = 150
plt.rcParams["font.size"] = 10


# ============================================================
# 1. Carregar resultados dos modelos clássicos
# ============================================================

results_ds1 = pd.read_csv("resultados_dataset1_final.csv", index_col=0)
results_ds2 = pd.read_csv("resultados_dataset2_final.csv", index_col=0)

# Garante que não tenha linha de GPT misturada
mask_gpt1 = ~results_ds1.index.str.contains("gpt", case=False)
mask_gpt2 = ~results_ds2.index.str.contains("gpt", case=False)
results_ds1 = results_ds1[mask_gpt1]
results_ds2 = results_ds2[mask_gpt2]

print("Modelos Dataset 1:", results_ds1.index.tolist())
print("Modelos Dataset 2:", results_ds2.index.tolist())


# ============================================================
# 2. Resultados do GPT (K-Fold) - preencha se mudar
# ============================================================

gpt_metrics = {
    "accuracy": 0.9492,
    "precision": 0.7326,
    "recall": 0.9786,
    "f1": 0.8379
}


# ============================================================
# 3. Gráfico geral de F1: todos os modelos (sem GPT)
# ============================================================

def plot_global_f1(results_ds1, results_ds2, filename="visao_geral_modelos.png"):
    fig, ax = plt.subplots(figsize=(10, 4))

    data = pd.DataFrame({
        "Dataset 1": results_ds1["f1"],
        "Dataset 2": results_ds2["f1"]
    }).sort_values("Dataset 1", ascending=False)

    x = np.arange(len(data))
    width = 0.35

    ax.bar(x - width/2, data["Dataset 1"], width, label="Dataset 1", edgecolor="black")
    ax.bar(x + width/2, data["Dataset 2"], width, label="Dataset 2", edgecolor="black")

    ax.set_ylabel("F1-Score", fontweight="bold")
    ax.set_title("Comparação de F1-Score entre Modelos e Datasets", fontsize=14, fontweight="bold")
    ax.set_xticks(x)
    ax.set_xticklabels(data.index, rotation=45, ha="right")
    ax.legend()
    ax.grid(axis="y", alpha=0.3)

    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close(fig)

plot_global_f1(results_ds1, results_ds2)


# ============================================================
# 4. Top 5 modelos por F1 (por dataset)
# ============================================================

def plot_top5(results, dataset_name, filename):
    fig, ax = plt.subplots(figsize=(6, 3))

    top5 = results.nlargest(5, "f1")
    colors = plt.cm.YlOrBr(np.linspace(0.4, 0.9, len(top5)))

    ax.barh(range(len(top5)), top5["f1"], color=colors, edgecolor="black")
    ax.set_yticks(range(len(top5)))
    ax.set_yticklabels(top5.index)
    ax.invert_yaxis()
    ax.set_xlabel("F1-Score")
    ax.set_title(f"Top 5 Modelos – {dataset_name}")

    for i, v in enumerate(top5["f1"]):
        ax.text(v + 0.002, i, f"{v:.4f}", va="center", fontsize=8)

    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close(fig)

plot_top5(results_ds1, "Dataset 1", "top5_dataset1.png")
plot_top5(results_ds2, "Dataset 2", "top5_dataset2.png")


# ============================================================
# 5. Trade-off Tempo vs F1 (por dataset)
# ============================================================

def plot_tradeoff(results, dataset_name, filename):
    fig, ax = plt.subplots(figsize=(6, 3))

    sc = ax.scatter(results["time"], results["f1"],
                    s=150,
                    c=np.arange(len(results)),
                    cmap="viridis",
                    edgecolor="black",
                    alpha=0.8)

    for name, row in results.iterrows():
        ax.annotate(name, (row["time"], row["f1"]),
                    fontsize=7, ha="left", va="bottom")

    ax.set_xlabel("Tempo total de CV (s)", fontweight="bold")
    ax.set_ylabel("F1-Score", fontweight="bold")
    ax.set_title(f"Trade-off Tempo vs Performance – {dataset_name}")
    ax.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close(fig)

plot_tradeoff(results_ds1, "Dataset 1", "tradeoff_dataset1.png")
plot_tradeoff(results_ds2, "Dataset 2", "tradeoff_dataset2.png")


# ============================================================
# 6. Métricas completas por modelo (Accuracy, Precision, Recall, F1)
# ============================================================

def plot_metrics_by_model(results, dataset_name, filename):
    fig, ax = plt.subplots(figsize=(8, 3))

    metrics = results[["accuracy", "precision", "recall", "f1"]].copy()
    metrics = metrics.sort_values("f1", ascending=False)

    idx = np.arange(len(metrics))
    width = 0.18

    ax.bar(idx - 1.5*width, metrics["accuracy"], width, label="Accuracy")
    ax.bar(idx - 0.5*width, metrics["precision"], width, label="Precision")
    ax.bar(idx + 0.5*width, metrics["recall"], width, label="Recall")
    ax.bar(idx + 1.5*width, metrics["f1"], width, label="F1-Score")

    ax.set_xticks(idx)
    ax.set_xticklabels(metrics.index, rotation=45, ha="right", fontsize=8)
    ax.set_ylim(0.7, 1.0)
    ax.set_title(f"Métricas por Modelo – {dataset_name}")
    ax.legend(fontsize=7)
    ax.grid(axis="y", alpha=0.3)

    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close(fig)

plot_metrics_by_model(results_ds1, "Dataset 1", "metricas_dataset1.png")
plot_metrics_by_model(results_ds2, "Dataset 2", "metricas_dataset2.png")


# ============================================================
# 7. Gráfico só do GPT (médias do K-Fold)
# ============================================================

def plot_gpt_metrics(gpt_metrics, filename="gpt_kfold_metrics.png"):
    fig, ax = plt.subplots(figsize=(4, 3))

    nomes = ["Accuracy", "Precision", "Recall", "F1-Score"]
    valores = [gpt_metrics["accuracy"],
               gpt_metrics["precision"],
               gpt_metrics["recall"],
               gpt_metrics["f1"]]

    ax.bar(nomes, valores, edgecolor="black")
    ax.set_ylim(0.0, 1.0)
    ax.set_ylabel("Score")
    ax.set_title("Desempenho do GPT (K-Fold)")

    for i, v in enumerate(valores):
        ax.text(i, v + 0.01, f"{v:.3f}", ha="center", fontsize=8)

    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close(fig)

plot_gpt_metrics(gpt_metrics)

def plot_metrics_heatmap(results, dataset_name, filename="heatmap_metricas.png",
                         gpt_metrics=None, gpt_label="GPT-4o (K-Fold)"):
    """
    Gera um heatmap de métricas para os modelos clássicos.
    Se gpt_metrics for fornecido, adiciona o GPT como mais uma linha.
    """
    metrics = results[["accuracy", "precision", "recall", "f1"]].copy()

    # Se quiser incluir o GPT neste heatmap
    if gpt_metrics is not None:
        metrics.loc[gpt_label] = [
            gpt_metrics["accuracy"],
            gpt_metrics["precision"],
            gpt_metrics["recall"],
            gpt_metrics["f1"],
        ]

    metrics = metrics.round(3)

    plt.figure(figsize=(8, 4))
    sns.heatmap(metrics, annot=True, cmap="YlGnBu", linewidths=0.5, fmt=".3f")
    plt.title(f"Heatmap de Métricas – {dataset_name}", fontsize=12, fontweight="bold")
    plt.ylabel("Modelos")
    plt.xlabel("Métricas")
    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close()


# ============================================================
# 11. Curva ROC dos modelos clássicos (somente modelos com predict_proba)
# ============================================================

from sklearn.metrics import roc_curve, auc

def plot_roc_curves(models_dict, X_test, y_test, filename="roc_curves.png", title="Curvas ROC"):
    """
    models_dict: {"Nome do Modelo": modelo_treinado}
    Apenas modelos com método predict_proba funcionarão.
    """

    plt.figure(figsize=(8, 5))

    # Converte rótulos para 0/1
    y_true = (y_test == "spam").astype(int)

    for name, model in models_dict.items():
        if hasattr(model, "predict_proba"):
            y_scores = model.predict_proba(X_test)[:, 1]
        elif hasattr(model, "decision_function"):
            y_scores = model.decision_function(X_test)
        else:
            continue  # ignora modelos sem score contínuo

        fpr, tpr, _ = roc_curve(y_true, y_scores)
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.3f})")

    plt.plot([0, 1], [0, 1], "k--", label="Aleatório (AUC = 0.5)")
    plt.xlabel("Taxa de Falsos Positivos (FPR)")
    plt.ylabel("Taxa de Verdadeiros Positivos (TPR)")
    plt.title(title)
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close()


# ============================================================
# 8. Comparação GPT vs Melhor Ensemble (Dataset 1)
# ============================================================

def plot_gpt_vs_best_ensemble(results_ds1, gpt_metrics,
                              filename="gpt_vs_ensemble.png"):
    # pega melhor ensemble por F1 no Dataset 1
    mask_ens = results_ds1.index.str.contains("Ensemble")
    ens_results = results_ds1[mask_ens]
    best_ensemble_name = ens_results["f1"].idxmax()
    best_ensemble = ens_results.loc[best_ensemble_name]

    modelos = [best_ensemble_name, "GPT-4o (K-Fold)"]
    metrics_names = ["accuracy", "precision", "recall", "f1"]

    fig, ax = plt.subplots(figsize=(6, 3))

    x = np.arange(len(metrics_names))
    width = 0.35

    ens_vals = [best_ensemble[m] for m in metrics_names]
    gpt_vals = [gpt_metrics[m] for m in metrics_names]

    ax.bar(x - width/2, ens_vals, width, label=best_ensemble_name, edgecolor="black")
    ax.bar(x + width/2, gpt_vals, width, label="GPT-4o (K-Fold)", edgecolor="black")

    ax.set_xticks(x)
    ax.set_xticklabels(["Accuracy", "Precision", "Recall", "F1-Score"])
    ax.set_ylim(0.0, 1.0)
    ax.set_ylabel("Score")
    ax.set_title("Comparação: Melhor Ensemble vs GPT")

    for i, v in enumerate(ens_vals):
        ax.text(i - width/2, v + 0.01, f"{v:.3f}", ha="center", fontsize=7)
    for i, v in enumerate(gpt_vals):
        ax.text(i + width/2, v + 0.01, f"{v:.3f}", ha="center", fontsize=7)

    ax.legend(fontsize=8)
    ax.grid(axis="y", alpha=0.3)

    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close(fig)

plot_gpt_vs_best_ensemble(results_ds1, gpt_metrics)


# ============================================================
# 9. Comparação F1 – todos os modelos + GPT (Dataset 1)
# ============================================================

def plot_all_models_with_gpt(results_ds1, gpt_metrics,
                             filename="todos_modelos_com_gpt.png"):
    df = results_ds1.copy()
    # tempo do GPT não foi medido aqui, colocamos NaN
    df.loc["GPT-4o (K-Fold)"] = {
        "accuracy": gpt_metrics["accuracy"],
        "precision": gpt_metrics["precision"],
        "recall": gpt_metrics["recall"],
        "f1": gpt_metrics["f1"],
        "roc_auc": np.nan,
        "time": np.nan
    }

    df_f1 = df["f1"].sort_values(ascending=False)

    fig, ax = plt.subplots(figsize=(8, 3))
    ax.bar(df_f1.index, df_f1.values, edgecolor="black")
    ax.set_ylabel("F1-Score")
    ax.set_ylim(0.0, 1.0)
    ax.set_title("Comparação de F1-Score – Modelos Clássicos + GPT (Dataset 1)")
    ax.set_xticklabels(df_f1.index, rotation=45, ha="right", fontsize=8)

    for i, v in enumerate(df_f1.values):
        ax.text(i, v + 0.01, f"{v:.3f}", ha="center", fontsize=7)

    plt.tight_layout()
    plt.savefig(filename, bbox_inches="tight")
    plt.close(fig)

plot_all_models_with_gpt(results_ds1, gpt_metrics)

plot_metrics_heatmap(
    results_ds1,
    "Dataset 1",
    "heatmap_dataset1.png",
    gpt_metrics=gpt_metrics,           
    gpt_label="GPT-4o (K-Fold)"
)

plot_metrics_heatmap(
    results_ds2,
    "Dataset 2",
    "heatmap_dataset2.png",
    gpt_metrics=gpt_metrics,           
    gpt_label="GPT-4o (K-Fold)"
)

plot_roc_curves(
    modelos_treinados_ds1,   # dicionário com os modelos já treinados
    X_test_ds1,
    y_test_ds1,
    filename="roc_dataset1.png",
    title="Curvas ROC – Dataset 1"
)

plot_roc_curves(
    modelos_treinados_ds2,
    X_test_ds2,
    y_test_ds2,
    filename="roc_dataset2.png",
    title="Curvas ROC – Dataset 2"
)



print("\nGráficos gerados com sucesso!")


Modelos Dataset 1: ['Logistic Regression', 'Decision Tree', 'Random Forest', 'SVM', 'Naive Bayes', 'XGBoost', 'Gradient Boosting', 'Ensemble (Voting)', 'Ensemble (Weighted)']
Modelos Dataset 2: ['Logistic Regression', 'Decision Tree', 'Random Forest', 'SVM', 'Naive Bayes', 'XGBoost', 'Gradient Boosting', 'Ensemble (Voting)', 'Ensemble (Weighted)']


C:\Users\xd-bo\AppData\Local\Temp\ipykernel_4744\3143752648.py:291: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(df_f1.index, rotation=45, ha="right", fontsize=8)



Gráficos gerados com sucesso!
